In [1]:
import os
import csv
import json_lines
from collections import Counter

In [2]:
import re
import spacy
import random
import json_lines
import json

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("ggplot")

In [4]:
input_data_dir = '/home/aorus/workspaces/simge/corpus/Sentence/Newindianexpress/'
all_data_path = os.path.join(input_data_dir,
                             '20181001-newindianexpress_sentence_classification_adjudicated_20181218.json')

In [5]:
all_data = []
with json_lines.open(all_data_path) as reader:
    for obj in reader:
        all_data.append(obj)
        
print('Number of total sentences:', len(all_data))

Number of total sentences: 8337


In [6]:
index_dict = {}
for row in all_data:
    
    doc_num, sent_num = row['sent_num'].split('-')
    
    sent_text = re.sub("http\S*\s", "", row['sentence'])
    sent_text = re.sub("((url)*\s*:\s*)*http\S*", "", sent_text)
    
    if not sent_text == "":
        
        sent = {'url': row['url'],
                'sent_num': row['sent_num'],
                'sentence': sent_text,
                'label': int(row['label'])}

        if doc_num not in index_dict:
            index_dict[doc_num] = {}

        index_dict[doc_num][sent_num] = sent

print('Number of articles:', len(index_dict.keys()))

Number of articles: 587


In [7]:
articles = []

for k, v in index_dict.items():
    
    article = {'id': 'a.' + k,
               'url' : v['1']['url'],
               'sentences': []}
    
    for _, sent in v.items():
        
        sent = {'sent_num': sent['sent_num'],
                'sentence': sent['sentence'],
                'label': sent['label']}
        
        article['sentences'].append(sent) 
    
    articles.append(article)

print('Number of articles:', len(articles))

Number of articles: 587


In [8]:
for article in articles:
    article['label_set'] = list({sent['label'] for sent in article['sentences']})

In [11]:
i = 0
for article in articles:
    if 2 in article['label_set'] and 1 not in article['label_set']:
        i += 1
print('Articles with only {0,2}:', i)

Articles with only {0,2}: 43


In [13]:
i = 0
for article in articles:
    if 1 not in article['label_set']:
        i += 1
print('Articles with at least 1 in label set:', len(articles) - i)

Articles with at least 1 in label set: 315


In [300]:
# Remove articles that does not contain any label '1'
articles = [article for article in articles if 1 in article['label_set']]
print('Number of articles:', len(articles))

Number of articles: 315


In [307]:
random.shuffle(articles)

In [308]:
# Split rates
test = 64
valid = 32
train = len(articles) - 64 - 32

In [309]:
thesis_train_data = articles[:train]
thesis_valid_data = articles[train:train+valid]
thesis_test_data = articles[train+valid:]

print('Number of articles;',
      '\nTrain\t:', len(thesis_train_data),
      '\nValid\t:', len(thesis_valid_data),
      '\nTest\t:',len(thesis_test_data))

Number of articles; 
Train	: 219 
Valid	: 32 
Test	: 64


In [310]:
thesis_train_label_list = [sent['label'] for art in thesis_train_data for sent in art['sentences']]
thesis_valid_label_list = [sent['label'] for art in thesis_valid_data for sent in art['sentences']]
thesis_test_label_list = [sent['label'] for art in thesis_test_data for sent in art['sentences']]

print('Number of sentences;',
      '\nTrain\t:', len(thesis_train_sent_list),
      '\nValid\t:', len(thesis_valid_sent_list),
      '\nTest\t:',len(thesis_test_sent_list))

Number of sentences; 
Train	: 4802 
Valid	: 1467 
Test	: 1527


In [311]:
tr_c = Counter(thesis_train_label_list)
va_c = Counter(thesis_valid_label_list)
te_c = Counter(thesis_test_label_list)

In [312]:
print('Sentence label distributions')

for v in [0, 1, 2]:
    
    print('\n-- Label: ', v, '--',    
          '\nTrain\t:', tr_c[v],
          '\nValid\t:', va_c[v],
          '\nTest\t:', te_c[v])

Sentence label distributions

-- Label:  0 -- 
Train	: 2094 
Valid	: 362 
Test	: 604

-- Label:  1 -- 
Train	: 955 
Valid	: 138 
Test	: 206

-- Label:  2 -- 
Train	: 45 
Valid	: 6 
Test	: 12


In [313]:
with open('Data/thesis_train_data.jsonl', 'w') as outfile:
    for article in thesis_train_data:
        json.dump(article, outfile)
        outfile.write('\n')
        
with open('Data/thesis_valid_data.jsonl', 'w') as outfile:
    for article in thesis_valid_data:
        json.dump(article, outfile)
        outfile.write('\n')
        
with open('Data/thesis_test_data.jsonl', 'w') as outfile:
    for article in thesis_test_data:
        json.dump(article, outfile)
        outfile.write('\n')